# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lategahn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lategahn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lategahn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# load data from database

engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable',engine)

#df.head()

X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    words = word_tokenize(text)
       
    words = [w for w in words if w not in stopwords.words("english")]

    lemmed_words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    
    return lemmed_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline_rfc = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_rfc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023E77581B80>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
# Print classification report on test data
def print_metrics(y_t,y_p):
    i = 0
    for col in y_t:
        print(col)
        print(classification_report(y_t[col], y_p[:, i]))
        i = i + 1

In [60]:
y_pred = pipeline_rfc.predict(X_test)


print_metrics(y_test,y_pred)

related
              precision    recall  f1-score   support

           0       0.65      0.13      0.22      1479
           1       0.79      0.98      0.87      5024
           2       0.43      0.12      0.18        51

    accuracy                           0.78      6554
   macro avg       0.62      0.41      0.42      6554
weighted avg       0.75      0.78      0.72      6554

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5451
           1       0.76      0.51      0.61      1103

    accuracy                           0.89      6554
   macro avg       0.83      0.74      0.78      6554
weighted avg       0.88      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6421
           1       0.62      0.29      0.40       133

    accuracy                           0.98      6554
   macro avg       0.80      0.64      0.69      6554
weighted avg       0.98      0.98      0.98      6554

other_weather
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6192
           1       0.50      0.15      0.24       362

    accuracy                           0.94      6554
   macro avg       0.72      0.57      0.60      6554
weighted avg       0.93      0.94      0.93      6554

direct_report
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      5311
           1       0.69      0.41      0.52      1243

    accuracy                           0.85      6554
   macro avg       0.78      0.68      0.71      6554
weighted avg       0.84      0.85      0.84   

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:

pipeline_rfc.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000023E77581B80>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000023E77581B80>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [17]:
parameters = {'clf__estimator__n_estimators': [50, 100, 200],
            'clf__estimator__min_samples_split': [2, 3, 4]}

cv = GridSearchCV(pipeline_rfc, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000023E77581B80>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 3, 4],
                         'clf__estimator__n_estimators': [50, 100, 200]})

In [26]:

y_pred_gs = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
print_metrics(y_test,y_pred_gs)

related
              precision    recall  f1-score   support

           0       0.69      0.40      0.51      1551
           1       0.83      0.94      0.88      4954
           2       0.45      0.53      0.49        49

    accuracy                           0.81      6554
   macro avg       0.66      0.62      0.63      6554
weighted avg       0.80      0.81      0.79      6554

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5441
           1       0.84      0.52      0.64      1113

    accuracy                           0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6219
           1       0.83      0.09      0.16       335

    accuracy                           0.95      6554
   macro avg       0.89      0.54      0.57      6554
weighted avg       0.95      0.95      0.93      6554

search_and_rescue
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6375
           1       0.67      0.03      0.06       179

    accuracy                           0.97      6554
   macro avg       0.82      0.52      0.53      6554
weighted avg       0.97      0.97      0.96      6554

security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6440
           1       0.33      0.01      0.02       114

    accuracy                           0.98      6554
   macro avg       0.66      0.50      0.50      6554
weighted avg       0.97      0.98      0.97    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [28]:
pipeline_abc = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline_abc.fit(X_train, y_train)


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023E77581B80>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [29]:
y_pred_abc = pipeline_abc.predict(X_test)

print_metrics(y_test,y_pred_abc)

related
              precision    recall  f1-score   support

           0       0.67      0.08      0.15      1551
           1       0.77      0.99      0.86      4954
           2       0.53      0.20      0.29        49

    accuracy                           0.77      6554
   macro avg       0.66      0.42      0.43      6554
weighted avg       0.74      0.77      0.69      6554

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5441
           1       0.77      0.56      0.64      1113

    accuracy                           0.90      6554
   macro avg       0.84      0.76      0.79      6554
weighted avg       0.89      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6265
           1       0.32      0.08      0.13       289

    accuracy                           0.95      6554
   macro avg       0.64      0.54      0.55      6554
weighted avg       0.93      0.95      0.94      6554

weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      4777
           1       0.86      0.66      0.74      1777

    accuracy                           0.88      6554
   macro avg       0.87      0.81      0.83      6554
weighted avg       0.88      0.88      0.87      6554

floods
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      6045
           1       0.85      0.58      0.69       509

    accuracy                           0.96      6554
   macro avg       0.91      0.79      0.84      6554
weighted avg       0.96      0.96      0.96      65

### 9. Export your model as a pickle file

In [32]:
pickle.dumps('classifier.pkl')

b'\x80\x04\x95\x12\x00\x00\x00\x00\x00\x00\x00\x8c\x0eclassifier.pkl\x94.'

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.